In [2]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [3]:
import os
import sys

In [4]:
os.chdir('../00_data/') # 存储数据的路径

In [5]:
ls

atec_anti_fraud_test_b.csv       df_50w.csv        submit_0212C.csv
atec_anti_fraud_test_b_demo.csv  df_test_50w.csv   submit_0212D.csv
atec_anti_fraud_train.csv        df_test.zip       train_head_10w.csv
atec_anti_fraud_train_demo.csv   submit_0211A.csv  train_modified.csv
df_10w.csv                       submit_0212A.csv
df_1m.csv                        submit_0212B.csv


# util

In [6]:
def get_col_null_rate(df, thres = 0.2):
    # 计算输入的df中各col null的比例
    # input  -- df of data
    # output -- df index and null rate
    
    info = pd.DataFrame()
    info['isnull'] = df.isnull().sum()
    info['null_ratio'] = info['isnull'] / df.shape[0]
    info = info.sort_values(by = 'null_ratio', ascending=False)
    info = info[info['null_ratio'] > thres]

    return info

In [7]:
def fill_col_null_with_mean(df):
    # 用于fill df中的NA
    # 另一种思路，直接用fillna中的阈值控制来填充，后续再改
    col_names = list(df.columns)
    for col in tqdm(col_names):
        if 'f' in col:  # to exclude: id date label 
            df[col] = df[col].transform(lambda x: x.fillna(x.mean()))
    return df

In [8]:
def clean_df(df, na_ratio=0.5):
    # na_ratio -- na比例
    # 1. 去除无标签的行
    # 2. 去除na比例大于na_ratio的列
    # 3. 其他na 按均值填充
    df_eff = df[df.label != -1]
    na_thres = int((1-0.5)*df.shape[0])
    df_dropna = df_eff.dropna(thresh = na_thres, axis =1)
    df_out = fill_col_null_with_mean(df_dropna)

    print('remove {} records without label.'.format(df.shape[0] - df_out.shape[0]))
    print('remove {} features where na ratio {}.'.format(df.shape[1] - df_out.shape[1], na_ratio))
    print(df_out.shape)
    return df_out

# Load and Clean data

- 行：去除标签-1的样本
- 列：NA比例超过0.5的列
- 填充其他的na

In [9]:
df = pd.read_csv('atec_anti_fraud_train_demo.csv')

In [ ]:
df_clean = clean_df(df)

  0%|          | 0/288 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
 85%|████████▍ | 244/288 [00:17<00:03, 14.66it/s]

In [ ]:
df_clean.to_csv('./df_1m.csv', index=False)  # 务必注意index=False，否则存储结果会多一列index

# convert test data

In [20]:
df_test = pd.read_csv('atec_anti_fraud_test_b_demo.csv')

In [21]:
train_keep_col = list(df_clean.columns.values)  # 得到train data保留的col name
train_keep_col.pop(1)  # test data中没有label 列

In [23]:
df_test = df_test.loc[:,train_keep_col]

In [24]:
df_test_clean = fill_col_null_with_mean(df_test)  # 填充测试集中的null

100%|██████████| 287/287 [00:06<00:00, 46.70it/s]


In [27]:
df_test_clean.to_csv('./df_test_50w.csv', index=False)